In [280]:
from music21 import *
from music21 import converter, instrument
import random
import networkx as nx
import numpy as np
import math

"""
Attempt 1.3 at Music Generation based off Midi Input
Difference from Attempt 1.2: Probability Transition Matrix for Note/Duration and Similarity Metrics
Author: Lauren Casey 2022
"""

In [281]:
"""
Finds shorter duration
"""
def findSmaller(og, gen):
    if(len(og) <= len(gen)):
        return og  
    else:
        return gen
    
    
"""
Finds the longer duration
"""
def findBigger(og, gen):
    if(len(og) > len(gen)):
        return og
    else:
        return gen
    

In [282]:
"""
Turns stream of music into array for easier use
"""
def turnStreamIntoArray(s):
    array = []
    for e in s.iter().notes:
        array.append(e.duration.quarterLength)
    return array

"""
Turns matrix into probability array for easier use
"""
def turnMatrixProbIntoArray(m):
    array = []
    i = 1
    
    print(m)
    print()
    while(i<len(m)):
        j = 1
        while(j<len(m)):
            array.append(m[i][j])
            j+=1
        i+=1
    
    print(array)
    return array

In [283]:
"""Calculates Cosine Similarity Duration"""
def cosineSimilarityDuration(original, generated):
    og = turnStreamIntoArray(original)
    gen = turnStreamIntoArray(generated)

    endHere = abs(len(og) - len(gen))
    if(len(og) > len(gen)):
        del og[:endHere]
    else:
        del gen[:endHere]

    dotProduct = np.dot(og, gen)
    magnitude = math.sqrt(dotProduct)
    
    return (dotProduct/magnitude)

In [284]:
"""Calculates Cosine Similarity Probability"""
def cosineSimilarityProbability(original, generated):
    og = turnMatrixProbIntoArray(durationMatrix(original))
    gen = turnMatrixProbIntoArray(durationMatrix(generated))
    
    dotProduct = np.dot(og ,gen)
    magnitude = math.sqrt(dotProduct)
    
    return (dotProduct/magnitude)

In [367]:
"""Calculates Earth Movers Distance"""
def EMD(original, generated):
    #Earth Movers Distance: two same size piles of dirt. Finds minimal cost of turning one pile into the other. 
    #Cost = amount of dirt being moved * distance moved
    
    smallerStream = findSmaller(original, generated)
    biggerStream  = findBigger(original, generated)
    
    i = 0
    earthMoversDistance = 0
    totalDistance = 0
    
    while(i<len(smallerStream)):
        #dirt = original[i].duration.quarterLength
        dirt = 1
        distance = abs(smallerStream[i].duration.quarterLength - biggerStream[i].duration.quarterLength)
        totalDistance += smallerStream[i].duration.quarterLength + biggerStream[i].duration.quarterLength
        earthMoversDistance += (dirt*distance)
        i+=1
        
        
    print("*******************************************************************************************************************")
    print("EMD - returns a number symbolizing minimal work to change one song into the other. \nSo in this case, how many changes to go from one song to the other. Therefore, the higher number, the more changes, the less similar.\nNote: **This is comparing each note at a time**")
    print("\nEarth Movers Distance: ", earthMoversDistance)
    print("EMD - Percent of Changes Needed: ", (earthMoversDistance/totalDistance)*100)
    return earthMoversDistance
        
    

In [368]:
"""Calculates Euclidean Distance"""
def euclideanDistance(original, generated):
    #how far off stream 1's durations are from stream 2's
    sumSquared = 0;
    
    for e in original.iter().notes:
        sumSquared += pow((e.duration.quarterLength + e.duration.quarterLength), 2)
    euDist = math.sqrt(sumSquared)
    return euDist

In [369]:
"""Calculates Euclidean Distance to compare Matrix Similarity"""
def similarMatricies(original, generated):
    #generate matrix off both streams, find how far off they (probability wise) are using Euclidean Distance
    matrixOG = durationMatrix(original)
    matrixGen = durationMatrix(generated)
    differenceAr = []
    
    i = 1
    j = 1
    euDist = 0
    
    while(i < len(matrixOG)):
        while(j < len(matrixGen)):
            euDist += pow((matrixOG[i][j] - matrixGen[i][j]), 2)
            j+=1
        i+=1
        
    return euDist
    
    
    
    

In [370]:
"""Calculates Edit Distance"""
def editDistance(originalStream, generatedStream, distance, pitch):
    #number of changes to get from original stream to generated stream
    smallerStream = findSmaller(originalStream, generatedStream)
    biggerStream = findBigger(originalStream, generatedStream)
    
    
    i = 0
    counter = 0
    if(distance):
        while (i < len(smallerStream.notes)):
            if(smallerStream.notes[i].duration.quarterLength != biggerStream.notes[i].duration.quarterLength):
                counter+=1
            i+=1
    print("*******************************************************************************************************************")
    print("Edit Distance: the higher the counter, the more changes needed, the less similar.\nNote: **This is comparing each note at a time**")
    print("\nEdit Distance: ", counter)
    print("Edit Distance - Percent Of Changes: ", (counter/len(smallerStream.notes))*100)
    print

        
    return counter

In [371]:
"""
Parameters: 
prev - previous note/duration chosen
matrix - note/duration matrix

Returns:
note/duration chosen to append to stream

Function: 
Chooses the next note/duration randomly
"""
def choose(prev, matrix):
    choosing = True;
    if prev == None:
        while(choosing):
            rowIndex = random.randrange(1, len(matrix)-1)
            colIndex = random.randrange(1, len(matrix)-1)
            if(matrix[rowIndex][colIndex] != 0.0):
                choosing = False;
                return matrix[0][colIndex]
    else:
        while(choosing):
            rowIndex = np.where(matrix[:, 0] == prev)[0]
            colIndex = random.randrange(1, len(matrix)-1)
            temp = ((matrix[rowIndex,:])[0])[colIndex]
            if(temp != 0.0):
                choosing = False;
                return matrix[0][colIndex]

In [379]:
"""
Parameters: 
horn - horn part in inputted music
DMatrix - matrix of duration probability transitions
D - stream of generated music
beatsToGen - number of beats to generate in the song

Returns:
Stream of generated music of durations

Function: 
Generates a specified length of music of durations based on input
"""

def generateDur(horn, DMatrix,D, beatsToGen):
    i = 0
    chosen = choose(None, DMatrix)
    D.append(note.Note(quarterLength = chosen))
    while i < beatsToGen:
        chosen = choose(chosen, DMatrix)
        
        if (chosen <= beatsToGen - i):
            D.append(note.Note(quarterLength = chosen))
            i+=chosen
            
            
    total = len(findSmaller(horn, D))
    #D.notesAndRests().show()
    
    print("*******************************************************************************************************************")
    print("Similarity Metrics")
    print("\nNote **Need total number for comparision of numbers...**\n")

    editDistance(horn, D, True, False)
    
    print("*******************************************************************************************************************")
    print("Euclidean Distance - the higher number it returns, the less similar.\nNote: **This is comparing one note at a time -- whereas the matrix ED is comparing the probability outcomes**")
    print("\nGenerated Matrix Euclidean Distance: ", similarMatricies(horn, D)*100)
    ED = euclideanDistance(horn, D)
    print("Euclidean Distance: ", ED)
    print("Euclidean Distance - Percent of Similarity", (1-(ED/total))*100)
    
    EMD(horn, D)
    
    print("*******************************************************************************************************************")
    print("Cosine Similarity - returns an angle. The smaller the angle, the more similar...")
    CS = cosineSimilarityDuration(horn, D)
    print("\nCosine Similarity (Duration): ", CS)
    print("Cosine Similarity Percentage of Similarity: ", (1-(CS/360))*100)
    #print("Cosine Similarity (Probability): ", cosineSimilarityProbability(horn, D))
    


In [380]:
"""
Parameters: 
horn - horn part in inputted music
NMatrix - matrix of note probability transitions
N - stream of generated music
beatsToGen - number of beats to generate in the song

Returns:
Stream of generated music of notes

Function: 
Generates a specified length of music of notes based on input
"""
def generateNote(horn, NMatrix,N, beatsToGen):
    i = 0
    while i < beatsToGen:
        chosen = choose(None, NMatrix)
        if(chosen == -1.0):
            N.append(note.Rest(quarterLength = 1))
        else:
            N.append(note.Note(pitch = chosen, quarterLength = 1))
        i+=1
    
            
    N.notesAndRests().show()

In [381]:

"""
Parameters: 
horn - horn part in inputted music

Returns:
Duration Matrix

Function: 
Matrix of probabilities of transitioning between durations in input music
"""
def durationMatrix(horn):
    #Creates matrix of duration changes
    matrix = np.array([[0]])
    prev = None
    for e in horn.iter().notesAndRests:#loop through each note and rest
        cur = e.duration.quarterLength
        if cur not in matrix[0]:#Adds column and row to matrix
            n,m = matrix.shape
            add = np.zeros((n, 1))
            add[0] = cur
            matrix = np.hstack((matrix, add))
            add = np.append(add, 0)
            matrix = np.vstack((matrix, add))
        if prev != None:#Increments matrix value
            prevInd = np.where(matrix[0] == prev)[0][0]
            curInd = np.where(matrix[0] == cur)[0][0]
            matrix[prevInd][curInd] = (matrix[prevInd][curInd]+1)
        prev = cur
    return matrix

In [382]:
"""
Parameters: 
horn - horn part in inputted music

Returns:
Note Matrix

Function: 
Matrix of probabilities of transitioning between notes in input music
"""
def noteMatrix(horn):
    #Creates matrix of duration changes
    matrix = np.array([[0]])
    prev = None
    for e in horn.iter().notesAndRests:#loop through each note
        if e.name == 'rest':
            cur = -1
        else:
            cur = e.pitch.frequency
        if cur not in matrix[0]:#Adds column and row to matrix
            n,m = matrix.shape
            add = np.zeros((n, 1))
            add[0] = cur
            matrix = np.hstack((matrix, add))
            add = np.append(add, 0)
            matrix = np.vstack((matrix, add))
        if prev != None:#Increments matrix value
            prevInd = np.where(matrix[0] == prev)[0][0]
            curInd = np.where(matrix[0] == cur)[0][0]
            matrix[prevInd][curInd] = (matrix[prevInd][curInd]+1)
        prev = cur
    return matrix

In [383]:
"""
Parameters: 
clef - Clef of Generated Music
key - Key of Generated Music
time - Time of Generated Music
title - Title of Generated Music

Returns:
Stream

Function: 
Creates a stream to append note/durations to, to create generated music
"""
def makeStream(clef, key, time, title):
    name = stream.Stream()
    name.append(clef)
    name.append(key)
    name.append(time)
    name.insert(0, metadata.Metadata())
    name.metadata.title = title
    
    return name
  

In [384]:
"""
Function:
Gets the song inputted and breaks it apart for song generation
"""

def songGrab():

#get Bach song
    chorals = corpus.search('bach', fileExtensions = 'xml')
    testSong = chorals[0].parse()
    
#grab time sig, key sig, and clef
    originalTimeSig = testSong.recurse().getElementsByClass(meter.TimeSignature)[0]
    originalKeySig = testSong.recurse().getElementsByClass(key.KeySignature)[0]
    originalClef = testSong.recurse().getElementsByClass(clef.Clef)[0]
    
#make some extra streams for etude output with the originals time sig, key sig, and clef
    D = makeStream(originalClef, originalKeySig, originalTimeSig, "Rhythm : Style Etude")
    N = makeStream(originalClef, originalKeySig, originalTimeSig, "Pitch : Style Etude")
    ND = makeStream(originalClef, originalKeySig, originalTimeSig, "Rhythm & Pitch : Style Etude")
    
    
#use horn part of chosen Bach song
    horn = testSong.parts[0].recurse()
    
#grab time sig - this is for counting up to the correct number of beats in measure when producing etude
    numer = horn.recurse().getElementsByClass(meter.TimeSignature)[0].numerator
#generate new etude

    beatsToGen = input("How many beats do you want to generate?: ")
    generateDur(horn, durationMatrix(horn),D,int(beatsToGen))
    #generateNote(horn, noteMatrix(horn), N, int(beatsToGen))
    

In [385]:
"""Initiate Process"""
songGrab()



How many beats do you want to generate?: 100
*******************************************************************************************************************
Similarity Metrics

Note **Need total number for comparision of numbers...**

*******************************************************************************************************************
Edit Distance: the higher the counter, the more changes needed, the less similar.
Note: **This is comparing each note at a time**

Edit Distance:  95
Edit Distance - Percent Of Changes:  76.61290322580645
*******************************************************************************************************************
Euclidean Distance - the higher number it returns, the less similar.
Note: **This is comparing one note at a time -- whereas the matrix ED is comparing the probability outcomes**

Generated Matrix Euclidean Distance:  4700.0
Euclidean Distance:  15.066519173319364
Euclidean Distance - Percent of Similarity 88.2292818958442